In [2]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import json


requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

url = "https://bdtd.ibict.br/vufind/api/v1/search"

In [3]:
stats = requests.get(url + "?lookfor=*%3A*&type=AllFields&facet[]=format&facet[]=institution&limit=0&filter[]=institution%3AUFRJ", verify=False).json()

print(f"Documentos: {stats['resultCount']}")
for facet in stats["facets"]["format"]:
    print(f"{facet['translated']}: {facet['count']}")

Documentos: 2832
Dissertação: 2205
Tese: 627


In [4]:
def get_json_data(metadata, n=100, fp=10):
    count = metadata["count"]
    records = []
    i = 0
    for i in range(0, count//n):
        current_url = url + metadata["href"].replace("limit=0", f"limit={n}") + f"&page={i+1}"
        req = requests.get(current_url, verify=False).json()
        print(req)
        records += req["records"]
        if i % fp == 0:
            print(current_url)
    if (count//n)*n < count:
        i += 1
        print("Last Index:", i+1)
        current_url = url + metadata["href"].replace("limit=0", f"limit={n}") + f"&page={i+1}"
        print("Last URL:", current_url)
        req = requests.get(current_url, verify=False).json()
        print(req)
        records += req["records"]
        
    return records

In [5]:
facets = stats["facets"]["format"]
facets

[{'value': 'masterThesis',
  'translated': 'Dissertação',
  'count': 2205,
  'href': '?limit=0&filter%5B%5D=institution%3A%22UFRJ%22&filter%5B%5D=format%3A%22masterThesis%22&lookfor=%2A%3A%2A&type=AllFields'},
 {'value': 'doctoralThesis',
  'translated': 'Tese',
  'count': 627,
  'href': '?limit=0&filter%5B%5D=institution%3A%22UFRJ%22&filter%5B%5D=format%3A%22doctoralThesis%22&lookfor=%2A%3A%2A&type=AllFields'}]

In [6]:
# Mestrados:
mast_meta = facets[0]

# Processando apenas os docs de mestrado
count = mast_meta["count"]
thesis_mast_meta = get_json_data(mast_meta, fp=20)
print(count, len(thesis_mast_meta))

# Doutorados:
doc_meta = facets[1]

# Processando apenas os docs de doutorados
count = doc_meta["count"]
thesis_doc_meta = get_json_data(doc_meta, fp=20)
print(count, len(thesis_doc_meta))

# unir os dois
thesis_meta = thesis_mast_meta + thesis_doc_meta
print(len(thesis_meta))
json_object = json.dumps(thesis_meta, indent = 2)

with open("thesis_meta.json", "w") as outfile:
    outfile.write(json_object)

{'resultCount': 2205, 'records': [{'id': 'UFRJ_e2f06e89e2445960208566d980f6c17a', 'title': 'Estevão Silva e Hélio Oiticica: brasilidade, a sensação revisitada', 'authors': {'primary': {'França, Cristina Pierre de': {'profile': [['NA']]}}}, 'contributors': {'advisor': {'Tavora, Maria Luisa Luz': {'profile': [['NA']]}}, 'referee': {'Cavalcanti, Ana Maria Tavares': {'profile': ['http://lattes.cnpq.br/3589319848375106']}, 'Ribeiro, Marcus Tadeu Daniel': {'profile': ['http://lattes.cnpq.br/6359526323861661']}}}, 'subjectsCNPQ': [['CNPQ::LINGUISTICA, LETRAS E ARTES::ARTES']], 'subjectsPOR': [['Arte brasileira']], 'institutions': ['UFRJ'], 'departements': ['Escola de Belas Artes'], 'programs': ['Programa de Pós-Graduação em Artes Visuais'], 'types': ['masterThesis'], 'accesslevel': 'openAccess', 'publicationDates': ['2002'], 'urls': ['http://hdl.handle.net/11422/5160'], 'formats': ['masterThesis'], 'languages': ['por']}, {'id': 'UFRJ_6550baea5ac50a368536a9a8aa69db49', 'title': 'Estevão Silva 